## Random sampling vs. stratified sampling
The purpose of this notebook is to build intuition about stratified sampling, by comparing it to random sampling. A guiding question should be

> For what ratio of parameters and realizations is the effect of stratified sampling noticeable and advantageous?

### Imports

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from scipy.stats import qmc

### Part I: Sampling a normal distribtuion using random vs. stratified sampling

The code snippet below compares two ways to sample from a normal distribution N(0, 1):

- Random sampling.
- Stratified sampling via Latin Hypercube Sampling (LHS) applied to uniform quantiles in [0, 1], then mapped to N(0, 1) using the inverse CDF (PPF).

The code snippet generates a plot that caintains the following:

- A histogram of purely random normal samples.
- A histogram of stratified samples (quantiles mapped through the normal PPF).
- The true normal probability density function.

Stratified sampling spreads samples more uniformly across the distribution’s quantiles, often producing a smoother approximation to the true PDF with fewer samples. To see this effect, experiment with the code below by varying the number of samples and the seed. With sufficiently many samples, both methods converge to the same distribution and their histograms will look increasingly similar.



In [ ]:
# Experiment parameters, please play around with these!
num_samples = 200
seed = 123


# Random normal sample
rng = np.random.default_rng(seed)
normal_values = rng.normal(loc=0.0, scale=1.0, size=num_samples)

# LHS quantiles in [0, 1], mapped to N(0,1) via inverse CDF (PPF)
sampler_1d = qmc.LatinHypercube(d=1, seed=seed)
quantiles = sampler_1d.random(n=num_samples).ravel()
lhs_values = sp.stats.norm.ppf(quantiles)

# Plot
fig, ax = plt.subplots(figsize=(6, 4))
ax.hist(normal_values, bins=30, alpha=0.5, density=True, label="Random N(0,1)")
ax.hist(
    lhs_values,
    bins=30,
    alpha=0.5,
    density=True,
    label="LHS via PPF",
    color="tab:orange",
)

x = np.linspace(-4, 4, 200)
ax.plot(x, sp.stats.norm.pdf(x), "k--", label="True PDF")

ax.set_title("Random vs stratified (LHS) sampling of N(0,1)")
ax.set_xlabel("Value")
ax.set_ylabel("Density")
ax.legend()
plt.tight_layout()
plt.show()

### Part II: Compare the space covrege of stratified sampling vs. random sampling

The code snippet below compares how well the two sampling strategies cover a high dimentional cube [0, 1]^d (with d = number_of_parameters) across repeated experiments:

- Random sampling: independent uniform draws in [0, 1]^d.
- Stratified sampling: Latin Hypercube Sampling (LHS) in [0, 1]^d.

For each of the number_of_experiments repetitions, the code draws realizations points for both methods and computes two coverage metrics:

- Minimum pairwise distance (“mindist” via qmc.geometric_discrepancy): the smallest Euclidean distance between any two sampled points. Larger values indicate points are more evenly spread (better space-filling), and smaller values indicate a clustering of points. 
- L2-star discrepancy (“L2-star” via qmc.discrepancy): A way to measure how much the samples deviate from uniform sampling. Smaller values indicate better uniform coverage of the space.

The code generates a figure with two histograms:

- Min pairwise distance in d dimensions: histograms of min(||x_i − x_j||) across experiments for Random vs. Stratified.
- L2-star discrepancy in d dimensions: histograms of the discrepancy metric across experiments for Random vs. Stratified.

What to expect:
- Modarate number of realizations with few number of parameters: Stratified sampling typically yields larger minimum pairwise distances and lower L2-star discrepancy than purely random sampling, reflecting more uniform, space-filling coverage, and less clustering. 
- Many parameters with few realizations (n << d): Both methods severely under-sample the space. Stratified sampling helps less in this regime, so the min-distance and discrepancy histograms often look similar and may overlap.
- As the number of realizations grows: Both methods converge toward uniform coverage, and their histograms become increasingly similar.

To explore the effects:

Vary number_of_parameters (dimensionality) and realizations (points per experiment), and adjust number_of_experiments for more stable histograms.

In [ ]:
def coverage_comparison(
    number_of_parameters: int = 30,
    realizations: int = 100,
    number_of_experiments: int = 100,
    seed: int = 123,
) -> None:
    
    min_dist_random = np.empty(number_of_experiments)
    min_dist_lhs = np.empty(number_of_experiments)
    l2star_random = np.empty(number_of_experiments)
    l2star_lhs = np.empty(number_of_experiments)

    for i in range(number_of_experiments):
        # Random uniform samples in [0, 1]^d, with d = number of parameters
        rng_random = np.random.default_rng(i)
        samples_random = rng_random.random(size=(realizations, number_of_parameters))
        min_dist_random[i] = qmc.geometric_discrepancy(samples_random, method="mindist")
        l2star_random[i] = qmc.discrepancy(samples_random, method="L2-star")

        # Latin Hypercube samples in [0, 1]^d, with d = number of parameters
        sampler = qmc.LatinHypercube(d=number_of_parameters, rng=rng_random)
        samples_lhs = sampler.random(realizations)
        min_dist_lhs[i] = qmc.geometric_discrepancy(samples_lhs, method="mindist")
        l2star_lhs[i] = qmc.discrepancy(samples_lhs, method="L2-star")

    # Plot histograms of the metrics
    _fig, axs = plt.subplots(1, 2, figsize=(10, 3))
    binning = "fd"  # Freedman–Diaconis rule

    axs[0].hist(min_dist_random, bins=binning, alpha=0.5, label="Random")
    axs[0].hist(min_dist_lhs, bins=binning, alpha=0.5, label="LHS")
    axs[0].set_title(f"Min pairwise distance in {number_of_parameters}D (higher is better)")
    axs[0].set_xlabel("min(||x_i - x_j||)")
    axs[0].set_ylabel("Count")
    axs[0].legend()

    axs[1].hist(l2star_random, bins=binning, alpha=0.5, label="Random")
    axs[1].hist(l2star_lhs, bins=binning, alpha=0.5, label="LHS")
    axs[1].set_title(f"L2-star discrepancy in {number_of_parameters}D (lower is better)")
    axs[1].set_xlabel("Discrepancy")
    axs[1].set_ylabel("Count")
    axs[1].legend()

    plt.tight_layout()
    plt.show()

In [ ]:
coverage_comparison(
    number_of_parameters=5, realizations=100, number_of_experiments=1000
)